## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [1]:
# 필요한 패키지 설치
!pip install -Uq python-dotenv langchain_teddynote langchain_openai langchain langchain-community faiss-cpu


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv("./.env", override=True)

True

In [4]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
LCEL


In [9]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [10]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [11]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [12]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [13]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### [실습] 셰프 프롬프트 템플릿 만들기

1. 변수 `food`에 음식 식재료를 입력받기 
2. `food` 재료로 만들 수 있는 음식 메뉴를 추천해주는 프롬프트 템플릿 객체 `chef_prompt` 생성하기
3. `food` 변수에 재료(값)를 넣어서 프롬프트를 완성해보기

In [15]:
# 실습 코드 작성
# 1. 변수 food에 음식 식재료 입력받기
food = input("식재료를 입력하세요: ")

# 2. 프롬프트 템플릿 객체 생성
chef_prompt = PromptTemplate(
    input_variables=["food"],
    template="당신은 요리사입니다. {food} 재료로 만들 수 있는 요리 메뉴 3가지를 추천해 주세요."
)

# 3. food 변수에 값 넣어서 프롬프트 완성
final_prompt = chef_prompt.format(food=food)

print("\n=== 생성된 프롬프트 ===")
print(final_prompt)


=== 생성된 프롬프트 ===
당신은 요리사입니다. 감자 재료로 만들 수 있는 요리 메뉴 3가지를 추천해 주세요.


### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [16]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000022A6C861850>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000022A6C861F10>, root_client=<openai.OpenAI object at 0x0000022A6C7F4410>, root_async_client=<openai.AsyncOpenAI object at 0x0000022A6C862330>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### [실습] 셰프 챗봇을 위한 openAI 모델 생성

1. 변수 `chef_model` 선언하기
2. 사용할 모델은 `gpt-4o-mini` 지정
3. 창의성을 조절하는 변수는 `0.1` 로 지정
4. 최대 활용할 토큰은 `4096`

In [18]:
# 실습 코드 작성
chef_model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=4096,
    temperature=0.1,
)
chef_model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000022A6C862F90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000022A6C8637D0>, root_client=<openai.OpenAI object at 0x0000022A6C861670>, root_async_client=<openai.AsyncOpenAI object at 0x0000022A6C862F00>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=4096)

### [실습] 셰프 챗봇을 위한 chef_chain 생성

- 프롬프트 템플릿 변수 `chef_prompt` 와 llm 모델을 담은 `chef_model` 모델을 체인(`|`)으로 연결하여 `chef_chain` 변수 생성

In [19]:
# 실습 코드 작성
chef_chain = chef_prompt | chef_model
chef_chain

PromptTemplate(input_variables=['food'], input_types={}, partial_variables={}, template='당신은 요리사입니다. {food} 재료로 만들 수 있는 요리 메뉴 3가지를 추천해 주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000022A6C862F90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000022A6C8637D0>, root_client=<openai.OpenAI object at 0x0000022A6C861670>, root_async_client=<openai.AsyncOpenAI object at 0x0000022A6C862F00>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=4096)

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [20]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [21]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

AIMessage(content='인공지능의 학습 방법은 크게 지도 학습, 비지도 학습, 강화 학습으로 나눌 수 있습니다.\n\n1. 지도 학습: 가장 일반적인 학습 방법으로, 레이블 된 데이터를 통해 모델을 학습시키는 방법입니다. 모델은 입력 데이터와 예측값 사이의 관계를 학습하여 새로운 데이터에 대한 예측을 할 수 있게 됩니다.\n\n2. 비지도 학습: 레이블이 없는 데이터를 사용하여 모델을 학습시키는 방법입니다. 데이터 간의 패턴이나 관계를 발견하거나 데이터를 그룹화하는데 사용됩니다. 대표적인 비지도 학습 알고리즘으로는 군집화나 차원 축소가 있습니다.\n\n3. 강화 학습: 에이전트가 환경과 상호 작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 에이전트는 시행착오를 통해 행동의 결과를 학습하고 학습된 지식을 바탕으로 최적의 행동을 선택하며 학습을 진행합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 349, 'prompt_tokens': 30, 'total_tokens': 379, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJrZmRxbiLQzpjlGg1Y9e5qwQV1ig', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--44a4bc56-807e-44c9-88b

In [22]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능의 학습 방법은 크게 지도학습, 비지도학습, 강화학습 세 가지로 나눌 수 있습니다.

1. 지도학습: 레이블된 데이터 셋을 기반으로 모델을 학습시키는 방법입니다. 모델은 입력 데이터와 정답 레이블 사이의 관계를 학습하여 새로운 입력 데이터에 대해 정확한 결과를 예측할 수 있습니다. 예를 들어, 고양이와 개를 구분하는 모델을 학습시킬 때, 이미지 데이터와 해당 이미지가 고양이인지 개인지에 대한 레이블을 제공하여 모델을 학습시킵니다.

2. 비지도학습: 레이블이 없는 데이터 셋을 기반으로 모델을 학습시키는 방법입니다. 모델은 데이터 간의 패턴을 찾아내거나 데이터를 그룹화하는 등의 작업을 수행합니다. 대표적으로 군집화(clustering)나 차원 축소 등이 있습니다.

3. 강화학습: 환경과 상호작용하며 행동을 통해 보상을 최대화하는 방향으로 모델을 학습시키는 방법입니다. 에이전트가 특정 환경에서 행동을 선택하고 그 결과에 따라 보상이 주어지며, 이를 통해 최적의 행동을 학습합니다. 딥마인드의 알파고가 강화학습을 사용하여 바둑을 배우는 과정이 대표적인 예시입니다.

인공지능은 이러한 학습 방법을 통해 데이터나 환경으로부터 패턴이나 최적의 행동을 학습하게 됩니다.

### 출력파서(Output Parser)


In [23]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [24]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000022A6C861850>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000022A6C861F10>, root_client=<openai.OpenAI object at 0x0000022A6C7F4410>, root_async_client=<openai.AsyncOpenAI object at 0x0000022A6C862330>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [25]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

'인공지능은 데이터를 입력받아 패턴을 학습하는 기술입니다. 기본적으로 학습 알고리즘이 데이터를 입력받고 입력된 데이터의 패턴을 분석하여 학습합니다. 이 학습된 데이터를 기반으로 예측하거나 결정을 내리는 작업을 수행하게 됩니다. 이러한 과정은 대부분의 경우 반복되며, 더 많은 데이터를 학습함으로써 더 나은 결과를 얻을 수 있습니다. 이러한 방식을 통해 인공지능은 점차적으로 더 정확한 예측이나 결정을 내릴 수 있도록 발전하게 됩니다.'

In [26]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능은 기계가 데이터를 학습하여 문제를 해결하는 능력을 의미합니다. 이를 가능하게 하는 학습 원리는 크게 규칙 기반 학습과 데이터 기반 학습으로 나눌 수 있습니다.

규칙 기반 학습은 사람들이 직접 규칙을 정의해 기계에게 가르치는 방식으로, 예를 들어 "만약 A 상황이라면 B 행동을 하라"와 같은 규칙을 만들어 기계에게 전달합니다. 그러나 이 방법은 복잡한 문제를 해결하기 어려우며, 모든 가능한 상황을 사람이 미리 정의해야 하는 한계가 있습니다.

반면 데이터 기반 학습은 기계에게 많은 양의 데이터를 제공하고, 기계가 이 데이터에서 패턴을 학습하여 문제를 해결하도록 하는 방식입니다. 기계는 데이터를 분석하고 통계적 모델링을 통해 패턴을 발견하고 예측하며, 이를 토대로 문제를 해결합니다. 이 방법은 복잡한 문제에 효과적이며, 학습 데이터의 양과 품질에 따라 성능이 좌우됩니다.

요약하자면, 인공지능의 학습 원리는 데이터를 통해 문제를 해결하는 방식으로, 규칙 기반 학습과 데이터 기반 학습으로 나뉘며, 데이터 기반 학습이 보다 효과적이고 널리 사용되고 있습니다.

### [실습] 최종! 셰프 챗봇을 실행해보기

- 마지막 StrOutputParser()까지 추가하여 `chef_chain` 완성하기
- 변수 `food_input`에 딕셔너리 형태로 재료 넣어 `chef_chain` 실행해보기
- Ex) 두부, 김치, 당근 ...

In [32]:
# 실습
chef_chain = chef_prompt | chef_model | StrOutputParser()

food_input = {"food": "두부"}

chef_answer = chef_chain.stream(food_input)
stream_response(chef_answer)

물론입니다! 두부를 활용한 맛있는 요리 메뉴 3가지를 추천해 드릴게요.

1. **두부 스테이크**  
   두부를 두껍게 썰어 소금, 후추로 간을 한 후 팬에 구워줍니다. 그 위에 간장, 마늘, 생강, 올리브유를 섞은 소스를 뿌려주면 맛있는 두부 스테이크가 완성됩니다. 채소와 함께 곁들여 내면 더욱 풍성한 한 끼가 됩니다.

2. **두부 김치찌개**  
   두부와 김치를 주재료로 한 찌개입니다. 먼저 돼지고기나 멸치로 육수를 내고, 김치를 넣어 볶은 후 두부를 큼직하게 썰어 넣습니다. 고춧가루와 간장으로 간을 맞추고 끓이면 깊고 진한 맛의 김치찌개가 완성됩니다.

3. **두부 샐러드**  
   부드러운 두부를 큐브 모양으로 썰어 신선한 채소(상추, 오이, 방울토마토 등)와 함께 섞습니다. 드레싱으로는 간장, 식초, 올리브유, 참깨를 섞어 만든 소스를 뿌려주면 상큼하고 건강한 두부 샐러드가 됩니다.

이 세 가지 요리는 모두 간단하면서도 맛있고 영양가가 높습니다. 즐거운 요리 시간 되세요!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [33]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [34]:
# 체인을 구성
chain = prompt | model | output_parser

In [35]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
Customer: Hi there! Can I see the menu, please?
Waiter: Of course! Here you go. Do you have any questions about the dishes?
Customer: Yes, I'm wondering what the chef's special is today.
Waiter: Today's special is grilled salmon with asparagus and lemon sauce.
Customer: That sounds delicious! I'll have that, please. And can I also get a side salad?
Waiter: Sure! Would you like the house dressing or something else?
Customer: I'll take the house dressing, please. Thank you!
Waiter: You're welcome! I'll bring that out for you shortly.

- 한글 해석:
고객: 안녕하세요! 메뉴를 보여주실 수 있나요?
웨이터: 물론이죠! 여기 있습니다. 요리에 대해 궁금한 점이 있으신가요?
고객: 네, 오늘의 셰프 스페셜이 무엇인지 궁금합니다.
웨이터: 오늘의 스페셜 메뉴는 아스파라거스와 레몬 소스와 함께 구운 연어입니다.
고객: 맛있을 것 같네요! 그걸로 할게요. 사이드 샐러드도 하나 주문할 수 있을까요?
웨이터: 네, 집에서 만든 드레싱을 원하시나요, 아니면 다른 걸 원하시나요?
고객: 집에서 만든 드레싱으로 해주세요. 감사합니다!
웨이터: 천만에요! 곧 가져다 드리겠습니다.


In [36]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
  - You: Hi! Just one, please.
  - Waiter: Great! Here’s the menu. Can I get you something to drink while you look?
  - You: Yes, I'll have a glass of water, please.
  - Waiter: Sure! Are you ready to order, or do you need a few more minutes?
  - You: I think I’m ready. I’d like the grilled chicken salad, please.
  - Waiter: Excellent choice! Would you like any dressing with that?
  - You: Yes, please. I'll have balsamic vinaigrette.
  - Waiter: Perfect! I’ll be right back with your order.

- 한글 해석:
  - 웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?
  - 당신: 안녕하세요! 저 혼자입니다.
  - 웨이터: 좋습니다! 여기에 메뉴가 있습니다. 보고 계시는 동안 음료를 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 웨이터: 알겠습니다! 주문하실 준비가 되셨나요, 아니면 몇 분 더 필요하신가요?
  - 당신: 준비가 된 것 같아요. 그릴에 구운 치킨 샐러드로 주문할게요.
  - 웨이터: 훌륭한 선택입니다! 드레싱은 필요하신가요?
  - 당신: 네, 발사믹 비네그레트를 주세요.
  - 웨이터: 완벽해요! 주문하신 대로 곧 가져다 드리겠습니다.

In [37]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hi there! I’d like to place an order for a pizza, please.
B: Sure! What size would you like – small, medium, or large?
A: I’ll take a large pizza. Can I get half pepperoni and half veggie, please?
B: Of course! Would you like any toppings or extra cheese?
A: Yes, please add extra cheese on the veggie side.
B: Great choice! Would you like anything to drink with that?
A: Yes, I’d like a large soda, please.
B: Alright! So, that’s one large pizza with half pepperoni, half veggie with extra cheese, and a large soda. Is that correct?
A: Yes, that’s right! How long will it take?
B: It should take about 30 minutes. Can I have your name for the order?
A: Sure, it’s Alex. Thank you!
B: Thank you, Alex! Your order will be ready soon.

- 한글 해석:
A: 안녕하세요! 피자를 주문하고 싶습니다.
B: 네, 물론입니다! 어떤 사이즈로 드릴까요 – 스몰, 미디엄, 라지 중에서요?
A: 라지 피자로 주세요. 반은 페퍼로니, 반은 베지로 주세요.
B: 알겠습니다! 추가 토핑이나 추가 치즈 필요하신가요?
A: 네, 베지 쪽에 추가 치즈 추가해 주세요.
B: 좋은 선택입니다! 음료는 또 필요하신가요?
A: 네, 큰 콜라 하나 주세요.
B: 알겠습니다! 그러면, 페퍼로니 반, 베지 반에 추가 치

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인
5. `Langsmith` 에 접속하여 실행 내용 확인

In [39]:
tour_guide_template = """
당신은 10년차 여행 가이드입니다. 주어진 여행갈 나라와 도시에서 3일간 가성비 여행 계획을 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 여행 계획:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(tour_guide_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

answer = chain.stream({"question": "한국, 서울 여행"})
stream_response(answer)

- 여행 계획:

### 1일차: 서울 중심 탐방
- **오전**
  - **경복궁** 방문: 한국 전통 궁궐 탐방. 외국인 관람객 무료 입장. (이용시간: 9:00~17:00)
  - **국립고궁박물관**: 경복궁 방문 후 바로 옆에 위치. 고궁의 역사와 문화에 대해 배울 수 있는 좋은 장소.
  
- **점심**
  - **인사동** 거리에서 전통 한식 맛보기: 식당이 많아 선택의 폭 넓음. 추천 메뉴: 비빔밥 또는 김치찌개.

- **오후**
  - **인사동 탐방**: 전통 찻집 및 기념품 가게 탐방. 예술적인 분위기의 거리에서 즐길 수 있는 다양한 공방 방문.
  - **북촌 한옥마을**: 전통 한옥이 줄지어 있는 골목 탐방. 사진 찍기 좋은 포인트가 많음.

- **저녁**
  - **홍대**에서 저렴한 길거리 음식 또는 맛집 탐방: 떡볶이, 순대 등이 유명.

### 2일차: 문화와 쇼핑
- **오전**
  - **명동** 탐방: 쇼핑 및 길거리 음식 (예: 프라푸치노, 핫도그) 즐기기.
  - **남산타워(N서울타워)**: 케이블카 이용하지 않고 도보로 올라가기. 전망대에서 서울 전경 감상. (입장료 할인 제공 여부 확인)

- **점심**
  - **명동**에서 점심: 돈까스 또는 짬뽕 추천.

- **오후**
  - **가로수길** 탐방: 트렌디한 카페와 상점 구경. 간단한 기념품 구매.
  - **현대백화점 또는 갤러리아백화점**: 쇼핑 및 쇼핑몰 내 다양한 먹거리 체험.

- **저녁**
  - **이태원**에서 외국 요리 또는 한국식 바비큐를 즐기기. 다양한 세계 각국의 음식 옵션이 있음.

### 3일차: 자연과 역사를 동시에
- **오전**
  - **서울숲**: 무료 입장, 휴식 및 산책하기 좋은 장소. 자전거 대여 가능.
  - **성수동** 탐방: 카페거리 및 아트 공간 탐방. 많은 아티스트들이 활동 중인 장소.

- **점심**
  - **성수동**의 로컬 음식점에서 간단한 점심: 샌드위치 또는 파스타.

- **오후**
  

In [40]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 대한민국 서울을 여행하고 싶습니다."})
# 스트리밍 출력
stream_response(answer)

- 여행 계획:

### 1일차: 서울의 전통과 현대를 만나다

**오전**
- **경복궁** 방문 (입장료: 무료, 전자 가이드: 3,000원)
    - 국민은행 앞에서 체험하는 전통 복식 대여 (5,000원)
- **광화문 근처**에서 전통 한국식 점심 (비빔밥 또는 국밥, 약 10,000원)

**오후**
- **북촌한옥마을** 탐방 (무료)
    - 오래된 한옥 건물과 전통 공예 상점 구경
- **삼청동 카페 거리**에서 커피 한 잔 (약 5,000원)

**저녁**
- **인사동**으로 이동하여 전통 기념품 쇼핑
- **스미스카페**에서 저녁 (약 15,000원)

---

### 2일차: 서울의 현대 문화를 즐기다

**오전**
- **남산서울타워** 방문 (환승 후 케이블카 왕복: 12,000원)
    - 서울 시내 전경 감상
- **한강공원**으로 이동하여 자전거 대여 (1시간: 3,000원)

**오후**
- **이태원**에서 점심 (다양한 국제 음식 옵션, 약 12,000원)
- **가로수길** 지역 탐방 (무료)
    - 유명한 브랜드 매장과 스타일리시한 카페들 방문

**저녁**
- **홍대** 거리로 이동하여 스트리트 공연 감상
- **홍대 주변 무료 나눔 먹거리**로 저녁 (밤에 도는 푸드트럭 - 약 10,000원)

---

### 3일차: 서울의 역사를 배우다

**오전**
- **서울역사박물관** 방문 (입장료: 무료)
    - 서울의 역사와 문화 관련 전시 관람
- **북악스카이웨이** 드라이브 및 전망 (무료, 차량 이용 시 유료 주차 고려)

**오후**
- **성북동**에서 한정식 점심 (약 15,000원)
- **조선일보사** 앞의 **추억의 거리** 산책 (무료)

**저녁**
- **동대문 디자인 플라자(DDP)** 야경 감상 및 디자이너 샵 탐방 (무료)
- **DDP 근처 푸드코트**에서 저녁 (약 10,000원)

### 총 예상 경비:
- 식비: 약 70,000원
- 교통비: 약 20,000원 (대중교

In [41]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 이탈리아 로마를 여행하고 싶습니다."})
# 스트리밍 출력
stream_response(answer)

- 여행 계획:

**여행지**: 이탈리아 로마  
**여행 기간**: 3일  

---

### **1일차: 고대 로마 탐방**

- **오전**  
  - **콜로세움**: 로마의 상징인 콜로세움을 방문합니다. 사전 예약을 통해 긴 대기 시간을 피하세요. (입장료: 약 18유로)
  - **로마 포럼**: 콜로세움 근처에 위치해 있으며, 고대 로마의 정치와 사회의 중심지였습니다. (포함된 티켓으로 입장 가능)

- **점심**  
  - **Trastevere** 지역의 현지 식당에서 파스타를 맛보세요. (예산: 약 15-20유로)

- **오후**  
  - **팔라티노 언덕**: 로마의 기원에 대해 배울 수 있는 곳으로, 아름다운 전망을 감상할 수 있습니다. (로마 포럼과 콜로세움 티켓 포함)
  - **베네치아 광장**: 로마의 대표적인 광장인 베네치아 광장에서 여유롭게 산책합니다.

- **저녁**  
  - **Trastevere 거리**의 현지 피자 가게에서 피자를 즐기세요. (예산: 약 10-15유로)

---

### **2일차: 바티칸과 예술 탐방**

- **오전**  
  - **바티칸 시국** 방문: 성 베드로 대성당 및 바티칸 박물관을 탐방합니다. 사전 예약 필수 (입장료: 약 30유로)
  - **성 베드로 광장**: 대성당 앞에서 여유롭게 사진 촬영.

- **점심**  
  - 바티칸 근처 카페에서 간단한 식사 (예산: 약 12-18유로)

- **오후**  
  - **판테온**: 로마의 고대 건축물 중 하나로, 무료로 입장할 수 있습니다.
  - **나보나 광장**: 로마의 아름다운 광장에서 커피를 즐기세요. (예산: 약 5-7유로)

- **저녁**  
  - 유명한 젤라토 가게에서 디저트를 즐기세요. (예산: 약 3-5유로)

---

### **3일차: 로마의 숨겨진 매력 탐방**

- **오전**  
  - **트라스테베레**: 이 지역의 독특한 분위기를 즐기며 아침 산책을 합니다.
  - **산타 마리아 인 트라스테